In [1]:
import os
import shutil
import glob
import pandas as pd
import random
import cv2

In [2]:
# Set the environment 
set_path = "/home/westray/Documents/Datasets/Original/SMD-Onshore"
targ_path = "/home/westray/Documents/Datasets/Original/SMD-Onshore/outputdata/target"
frame_path = "/home/westray/Documents/Datasets/Original/SMD-Onshore/inputdata/frames"
subset = "MVI_1469_VIS"

In [3]:
# class name can't be numeric 
# class_dic = {3.0: "boat1", 4.0:"boat2"}
class_dic = {1.0:"ferry", 2.0: "buoy", 3.0:"vessel",4.0:"speedboat", 
             5.0: "boat", 6.0:"kayak",7.0:"sail", 8.0: "other", 9.0:"bird/plane",10.0:"other", 0.0: "other"}


In [ ]:
iminput = "/home/westray/Documents/Datasets/Original/SMD-Onshore/outputdata/target"
data_dirs = {
    "smd": iminput
}
root = set_path
img_out_dir = "/home/westray/Documents/Datasets/Preprocessed/SMD-TLT/images/"
labels_out_dir = "/home/westray/Documents/Datasets/Preprocessed/SMD-TLT/labels/"
!rm -r $img_out_dir
!rm -r $labels_out_dir
!mkdir -p $img_out_dir
!mkdir -p $labels_out_dir

for dataset_name, root in data_dirs.items():
#     annot_file = os.path.join(root, 'MVI_0799_VIS_OB.txt')
    annot_list = glob.glob(os.path.join(root, "*.txt"))
    annot_list = [os.path.join(targ_path, subset+".txt")]
    for p in annot_list:
        # Read binary annotations
        annot_df = pd.read_csv(p, header=0) # index_col=None
        annot_df.columns = ['path', 'xmin', 'ymin', 'xmax', 'ymax', 'class']
        annot_df = annot_df.drop_duplicates()
        # Format columns according to COCO format
        coco = ['class', 'xmin', 'ymin', 'xmax', 'ymax']
        for img_filename, annotations in annot_df.groupby('path'):
            df = annotations[coco]
            df_trans = df
#             print(df)
            img_name, img_ext = os.path.splitext(img_filename)
    #         base_name = f"{dataset_name}_{img_name}"
            annot_name = p.split("/")[-1].split(".")[0]
            img_out_path = os.path.join(img_out_dir, img_name.split("/")[-1]+ annot_name +img_ext)
            annot_out_path = os.path.join(labels_out_dir, img_name.split("/")[-1] + annot_name+'.txt')
            root = set_path
            # Copy the image
            img_src = os.path.join(root, img_filename)
            im = cv2.imread(img_src)
            im_shape = im.shape
            df_trans.loc[:,'class'] = df.loc[:,'class']
            df_trans.loc[:,'xmin'] = (df.loc[:,'xmin']+(df.loc[:,'xmax']-df.loc[:,'xmin'])/2)/im_shape[1]
            df_trans.loc[:,'ymin'] = (df.loc[:,'ymin']+(df.loc[:,'ymax']-df.loc[:,'ymin'])/2)/im_shape[0]
            df_trans.loc[:,'xmax'] = (df.loc[:,'xmax']-df.loc[:,'xmin'])/im_shape[1]
            df_trans.loc[:,'ymax'] = (df.loc[:,'ymax']-df.loc[:,'ymin'])/im_shape[0]            
#             try:
            shutil.copyfile(img_src, img_out_path)
            df_trans.to_csv(annot_out_path, header=False, index=False, sep=" ")
#             except Exception:
#                 pass
            # Create the annotations file
        print(f"Wrote KITTI dataset for original dataset {annot_name}.")